In [17]:
import numpy as np
import glob
import re
import os

import KID_S21 as S21

import matplotlib.pyplot as plt

files = sorted(glob.glob('Test_S21/KID*.dat'))

kid = []
for file in files:
    kid.append(int(re.findall("KID(\d+)_\d{3}dBm_", file)[0]))

    

In [2]:
## logbook

# - in data if datarange in freq is below or above a certain range we know that there is an issue...
# - add internal power
# - min and max temperature
# - smaller fit range



In [3]:
[files, kids] = S21.find_S21_files('Test_S21/')

In [4]:
import re

def extract_data_by_temperature(file_contents):
    data_by_temperature = {}
    current_temperature = None
    current_data = []

    for line in file_contents:
        # Check if the line contains temperature information
        temp_match = re.match(r'Temperature in K:(\d+\.\d+)', line)
        if temp_match:
            #print(f"Found temperature: {temp_match.group(1)}")
            if current_temperature is not None and current_data:
                # Store the current data block before moving to the next temperature
                data_by_temperature[current_temperature] = current_data
            
            # Start a new temperature section
            current_temperature = float(temp_match.group(1))
            current_data = []
        
        # Check if the line contains tab-separated data (GHz, dB, Rad)
        elif re.match(r'\d+\.\d+\t', line):
            current_data.append(line.strip().split('\t'))
    
    # Save the last block of data
    if current_temperature is not None and current_data:
        data_by_temperature[current_temperature] = current_data

    return data_by_temperature

In [6]:
import pandas as pd
import re

def preprocess_file(file_contents):
    data_by_temperature = {}
    current_temperature = None
    current_data = []

    for line in file_contents:
        # Check if the line contains temperature information
        temp_match = re.match(r'Temperature in K:(\d+\.\d+)', line)
        if temp_match:
            if current_temperature is not None and current_data:
                # Convert current_data list to a pandas DataFrame and store it
                df = pd.read_csv(pd.compat.StringIO("\n".join(current_data)), sep='\t', header=None, names=['Frequency', 'dB', 'Rad'])
                data_by_temperature[current_temperature] = df
            
            # Start a new temperature section
            current_temperature = float(temp_match.group(1))
            current_data = []
        
        # Check if the line contains tab-separated data (GHz, dB, Rad)
        elif re.match(r'\d+\.\d+\t', line):
            current_data.append(line.strip())
    
    # Save the last block of data
    if current_temperature is not None and current_data:
        df = pd.read_csv(pd.compat.StringIO("\n".join(current_data)), sep='\t', header=None, names=['Frequency', 'dB', 'Rad'])
        data_by_temperature[current_temperature] = df

    return data_by_temperature

# # Read the file
# file_path = '/path/to/your/KID1_102dBm_.dat'

# with open(file_path, 'r') as file:
#     file_contents = file.readlines()

# # Preprocess the file and store the data organized by temperature
# data_by_temperature = preprocess_file(file_contents)

In [7]:
import pandas as pd
import re
import io

def preprocess_file_fast(file_contents):
    data_by_temperature = {}
    current_temperature = None
    current_data = []

    # Process each line, reducing Python overhead
    for line in file_contents:
        # If temperature is found
        temp_match = re.match(r'Temperature in K:(\d+\.\d+)', line)
        if temp_match:
            if current_temperature is not None and current_data:
                # Convert current_data to a pandas DataFrame
                data_str = "\n".join(current_data)
                df = pd.read_csv(io.StringIO(data_str), sep='\t', header=None, names=['Frequency', 'dB', 'Rad'])
                data_by_temperature[current_temperature] = df
            
            # Set new temperature and reset current data
            current_temperature = float(temp_match.group(1))
            current_data = []
        
        # Collect tab-separated data (GHz, dB, Rad)
        elif re.match(r'\d+\.\d+\t', line):
            current_data.append(line.strip())
    
    # Save the last block of data
    if current_temperature is not None and current_data:
        data_str = "\n".join(current_data)
        df = pd.read_csv(io.StringIO(data_str), sep='\t', header=None, names=['Frequency', 'dB', 'Rad'])
        data_by_temperature[current_temperature] = df

    return data_by_temperature

In [8]:
# Read the file
file_path = 'Test_S21_2D/KID1_126dBm_.dat'

with open(file_path, 'r') as file:
    file_contents = file.readlines()

# Extract data organized by temperature
data_by_temperature = preprocess_file_fast(file_contents)

In [24]:
all_temperatures = list(data_by_temperature.keys())

In [27]:
plot_temperatures = all_temperatures[::10]
plot_temperatures

[0.021563,
 0.039716,
 0.060305,
 0.080961,
 0.101531,
 0.122612,
 0.143989,
 0.165384]

In [20]:
# Directory to save CSV files
output_dir = "temperature_data"
os.makedirs(output_dir, exist_ok=True)

for temperature, df in data_by_temperature.items():
    #print(f"Temperature: {temperature} K")

    # Extract Frequency and S21 data
    frequencies = df['Frequency'].values
    s21_values = df['dB'].values
    rad_values = df['Rad'].values
    
    file_name = f"KID_P_{temperature*1e3:.2f}mK.csv"
    output_path = os.path.join(output_dir, file_name)

    # Save the DataFrame (Frequency, S21, Rad) to CSV
    df.to_csv(output_path, index=False)

    print(f"Data for {temperature} K saved to {output_path}")

Data for 0.021563 K saved to temperature_data/KID_P_21.56mK.csv
Data for 0.021287 K saved to temperature_data/KID_P_21.29mK.csv
Data for 0.023335 K saved to temperature_data/KID_P_23.34mK.csv
Data for 0.025376 K saved to temperature_data/KID_P_25.38mK.csv
Data for 0.027461 K saved to temperature_data/KID_P_27.46mK.csv
Data for 0.029501 K saved to temperature_data/KID_P_29.50mK.csv
Data for 0.031523 K saved to temperature_data/KID_P_31.52mK.csv
Data for 0.033556 K saved to temperature_data/KID_P_33.56mK.csv
Data for 0.035669 K saved to temperature_data/KID_P_35.67mK.csv
Data for 0.037725 K saved to temperature_data/KID_P_37.73mK.csv
Data for 0.039716 K saved to temperature_data/KID_P_39.72mK.csv
Data for 0.041859 K saved to temperature_data/KID_P_41.86mK.csv
Data for 0.043886 K saved to temperature_data/KID_P_43.89mK.csv
Data for 0.045908 K saved to temperature_data/KID_P_45.91mK.csv
Data for 0.047902 K saved to temperature_data/KID_P_47.90mK.csv
Data for 0.050049 K saved to temperature

In [21]:
temperature

0.167567

In [13]:


# Loop over each temperature and save to CSV
for temperature, df in data_by_temperature.items():
    # Create a filename based on the temperature
    file_name = f"temperature_{temperature:.6f}K.csv"
    output_path = os.path.join(output_dir, file_name)

    # Save the DataFrame (Frequency, S21, Rad) to CSV
    df.to_csv(output_path, index=False)

    print(f"Data for {temperature} K saved to {output_path}")


array([2.458218, 2.458218, 2.458218, ..., 2.458218, 2.458218, 2.458218])

In [9]:
# Loop over each temperature and get the corresponding data

f_list = []
S21_dB_list = []

for temperature, temp_data in data_by_temperature.items():
    #print(f"Temperature: {temperature} K")
    
    f = []
    S21_dB = []
    S21_rad = []
    
    # Print the data (Frequency, dB, Rad) for this temperature
    for row in temp_data:
        freq, db, rad = row
        
        f.append(float(freq))
        S21_dB.append(float(db))
        S21_rad.append(float(rad))
        
        S21_data = np.array([f, S21_dB, S21_rad])
        f = None
        S21_dB = None
        S21_rad = None
        
        #np.savetxt("test.csv", S21_data, delimiter = ",")

ValueError: too many values to unpack (expected 3)

In [10]:
row

'Frequency'

In [ ]:
fig, ax = plt.subplots()
ax.plot(f_list[0], S21_dB_list[0])

In [ ]:
fig, ax = plt.subplots()
ax.plot(f, S21_dB)

In [ ]:
f

In [ ]:
data_by_temperature.items()

In [ ]:
# Print a list of all temperatures
all_temperatures = list(data_by_temperature.keys())
print(all_temperatures)

In [ ]:
all_temperatures = list(test.keys())
print(all_temperatures)

In [ ]:
path = 'S21_files/'
files = sorted(glob.glob(path + 'KID*.dat')) # a sorted lis of filenames in path
nr_files = len(files)

kid = []
for file in files:
    kid.append(int(re.findall("KID(\d+)_\d{3}dBm_", file)[0]))

kids = np.unique(kid)

In [ ]:
files[0]